# Divergent data reduction for Amor

In this notebook, we will look at how to use the `essreflectometry` package with Sciline,
for reflectometry data collected from the PSI instrument [Amor](https://www.psi.ch/en/sinq/amor) in [divergent beam mode](https://www.psi.ch/en/sinq/amor/selene).

This notebook shows the basics of computing reduced results for a set of runs and saving them to Orso file format.

## Setup

In [ ]:
# %matplotlib widget
import warnings
import scipp as sc
from ess import amor
from ess.amor import data  # noqa: F401
from ess.reflectometry.types import *
from ess.amor.types import *
from ess.reflectometry import batch_compute

# The files used in this tutorial have some issues that makes scippnexus
# raise warnings when loading them. To avoid noise in the notebook the warnings are silenced.
warnings.filterwarnings('ignore', 'Failed to convert .* into a transformation')
warnings.filterwarnings('ignore', 'Invalid transformation')
warnings.filterwarnings('ignore', 'invalid value encountered')

## Create and configure the workflow

We begin by creating the Amor workflow object which is a skeleton for reducing Amor data,
with pre-configured steps, and then set the missing parameters which are specific to each experiment:

In [ ]:
workflow = amor.AmorWorkflow()
workflow[SampleSize[SampleRun]] = sc.scalar(10.0, unit='mm')
workflow[SampleSize[ReferenceRun]] = sc.scalar(10.0, unit='mm')

workflow[ChopperPhase[ReferenceRun]] = sc.scalar(-7.5, unit='deg')
workflow[ChopperPhase[SampleRun]] = sc.scalar(-7.5, unit='deg')

workflow[WavelengthBins] = sc.geomspace('wavelength', 2.8, 12.5, 2001, unit='angstrom')

# The YIndexLimits and ZIndexLimits define ranges on the detector where
# data is considered to be valid signal.
# They represent the lower and upper boundaries of a range of pixel indices.
workflow[YIndexLimits] = sc.scalar(11), sc.scalar(41)
workflow[ZIndexLimits] = sc.scalar(80), sc.scalar(370)
workflow[BeamDivergenceLimits] = (
    sc.scalar(-0.75, unit='deg'),
    sc.scalar(0.75, unit='deg'),
)

## Setting the reference run

The reference represents the intensity reflected by the super-mirror.
The same run is used for normalizing all sample runs,
and it thus only needs to be reduced once.

In this subsection, we reduce the reference run and cache it onto the workflow to speed-up subsequent processing.

In [ ]:
workflow[Filename[ReferenceRun]] = amor.data.amor_run(614)

# The sample rotation value in the file is slightly off, so we set it manually
workflow[SampleRotationOffset[ReferenceRun]] = sc.scalar(0.05, unit='deg')

# Set the result back onto the pipeline to cache it
workflow[ReducedReference] = workflow.compute(ReducedReference)

## Computing sample reflectivity from batch reduction

We now compute the sample reflectivity from 4 runs that used different sample rotation angles.
The measurements at different rotation angles cover different ranges of $Q$.

We use the `batch_compute` function which makes it easy to process multiple runs at once.

In [ ]:
runs = {
    '608': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(608),
    },
    '609': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(609),
    },
    '610': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(610),
    },
    '611': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(611),
    },
}

# Compute R(Q) for all runs
r_of_q = batch_compute(workflow, runs, target=ReflectivityOverQ)
r_of_q

In [ ]:
sc.plot(r_of_q.hist(), norm='log', vmin=1e-5)

## Scaling the reflectivity curves to overlap

In case we know the curves are have been scaled by different factors (that are constant in Q) it can be useful to scale them so they overlap:

In [ ]:
scaled_r = batch_compute(
    workflow,
    runs,
    target=ReflectivityOverQ,
    scale_to_overlap=(
        sc.scalar(0.01, unit='1/angstrom'),
        sc.scalar(0.014, unit='1/angstrom'),
    ),
)

sc.plot(scaled_r.hist(), norm='log', vmin=1e-5)

## Save data

We can save the computed $I(Q)$ to an [ORSO](https://www.reflectometry.org) [.ort](https://github.com/reflectivity/file_format/blob/master/specification.md) file using the [orsopy](https://orsopy.readthedocs.io/en/latest/index.html) package.

First, we need to collect the metadata for that file.
To this end, we insert a parameter to indicate the creator of the processed data.

In [ ]:
from ess.reflectometry import orso
from orsopy import fileio

In [ ]:
workflow[orso.OrsoCreator] = orso.OrsoCreator(
    fileio.base.Person(
        name='Max Mustermann',
        affiliation='European Spallation Source ERIC',
        contact='max.mustermann@ess.eu',
    )
)

We build our ORSO dataset from the computed $I(Q)$ and the ORSO metadata:

In [ ]:
iofq_datasets = batch_compute(
    workflow,
    runs,
    target=orso.OrsoIofQDataset,
    scale_to_overlap=(
        sc.scalar(0.01, unit='1/angstrom'),
        sc.scalar(0.014, unit='1/angstrom'),
    ),
)

We also add the URL of this notebook to make it easier to reproduce the data:

In [ ]:
for ds in iofq_datasets.values():
    ds.info.reduction.script = (
        'https://scipp.github.io/essreflectometry/user-guide/amor/amor-reduction-simple.html'
    )

Finally, we can save the data to a file.
Note that `iofq_datasets` contains [orsopy.fileio.orso.OrsoDataset](https://orsopy.readthedocs.io/en/latest/orsopy.fileio.orso.html#orsopy.fileio.orso.OrsoDataset)s.

In [ ]:
fileio.orso.save_orso(
    datasets=list(iofq_datasets.values()), fname='amor_reduced_iofq.ort'
)

Look at the first 50 lines of the file to inspect the metadata:

In [ ]:
!head amor_reduced_iofq.ort -n50